In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import glob
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
zip_dir = tf.keras.utils.get_file(origin=_URL,
                                  fname = 'flower_photos.tgz',
                                  extract = True)
base_dir = os.path.join(os.path.dirname(zip_dir), 'flower_photos')

In [3]:
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

In [6]:
for cl in classes:
    img_path = os.path.join(base_dir, cl)
    images = glob.glob(img_path+'/*.jpg')
    print('{}:{} images'.format(cl, len(images)))
    train, val = images[:round(len(images)*0.8)], images[round(len(images)*0.8):]
    

    for t in train:
      if not os.path.exists(os.path.join(base_dir, 'tra', cl)):
          os.makedirs(os.path.join(base_dir, 'tra', cl))
      shutil.move(t, os.path.join(base_dir, 'tra', cl))

    for v in val:
      if not os.path.exists(os.path.join(base_dir, 'vali', cl)):
          os.makedirs(os.path.join(base_dir, 'vali', cl))
      shutil.move(v, os.path.join(base_dir, 'vali', cl))

roses:128 images
daisy:633 images
dandelion:898 images
sunflowers:699 images
tulips:799 images


In [7]:
train_dir = os.path.join(base_dir, 'tra')
val_dir = os.path.join(base_dir, 'vali')

In [8]:
batch_size = 100
image_size = 150

In [9]:
train_image_generator = ImageDataGenerator(rescale=1./255,
                                           rotation_range = 45,
                                           width_shift_range=0.15,
                                           height_shift_range = 0.15,
                                           zoom_range = 0.5,
                                           fill_mode='nearest',
                                           horizontal_flip = True)

val_image_generator = ImageDataGenerator(rescale=1./255)


In [10]:
train_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                          shuffle = True,
                                                          target_size=(image_size, image_size),
                                                          directory = train_dir,
                                                          class_mode = 'sparse')

val_gen = val_image_generator.flow_from_directory(batch_size=batch_size,
                                                      shuffle = False,
                                                      target_size=(image_size, image_size),
                                                      directory = val_dir,
                                                      class_mode = 'sparse')

Found 3037 images belonging to 5 classes.
Found 633 images belonging to 5 classes.


In [11]:
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3), padding='same',activation = 'relu', input_shape = (150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3), padding='same', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [12]:
model.compile(optimizer='Adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
epochs = 80
history = model.fit_generator (train_gen,
                              epochs = epochs,
                              steps_per_epoch = np.ceil(train_gen.n/batch_size),
                              validation_data = val_gen,
                              validation_steps = np.ceil(val_gen.n/batch_size))

<ipython-input-14-80edcd2e79ea>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator (train_gen,


Epoch 1/80


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


31/31 [==============================] - 103s 3s/step - loss: 1.5530 - accuracy: 0.3224 - val_loss: 1.2990 - val_accuracy: 0.4250
Epoch 2/80
31/31 [==============================] - 96s 3s/step - loss: 1.1942 - accuracy: 0.4867 - val_loss: 1.0843 - val_accuracy: 0.5972
Epoch 3/80
31/31 [==============================] - 97s 3s/step - loss: 1.0786 - accuracy: 0.5634 - val_loss: 1.1916 - val_accuracy: 0.5703
Epoch 4/80
31/31 [==============================] - 98s 3s/step - loss: 1.0198 - accuracy: 0.5868 - val_loss: 0.8963 - val_accuracy: 0.6667
Epoch 5/80
31/31 [==============================] - 96s 3s/step - loss: 0.9827 - accuracy: 0.6085 - val_loss: 0.9463 - val_accuracy: 0.6445
Epoch 6/80
31/31 [==============================] - 96s 3s/step - loss: 0.9456 - accuracy: 0.6276 - val_loss: 0.8093 - val_accuracy: 0.6983
Epoch 7/80
31/31 [==============================] - 96s 3s/step - loss: 0.9065 - accuracy: 0.6490 - val_loss: 0.8628 - val_accuracy: 0.6524
Epoch 8/80
31/31 [============